In [67]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

In [68]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [69]:
i1 = base.loc[base.price <= 10]
base.price.mean()
base = base[base.price > 10]
i2 = base.loc[base.price > 350000]
base = base.loc[base.price < 350000]

In [70]:
base.loc[pd.isnull(base['vehicleType'])]
base['vehicleType'].value_counts() #limousine     
base.loc[pd.isnull(base['gearbox'])]
base['gearbox'].value_counts() #Manual
base.loc[pd.isnull(base['model'])]
base['model'].value_counts() #Golf
base.loc[pd.isnull(base['fuelType'])]
base['fuelType'].value_counts() #Gasolina
base.loc[pd.isnull(base['notRepairedDamage'])]
base['notRepairedDamage'].value_counts() #Nein

notRepairedDamage
nein    259301
ja       34004
Name: count, dtype: int64

In [71]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}

In [72]:
base = base.fillna(value = valores)

In [73]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [74]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

In [75]:
categorical_columns = [0, 1, 3, 5, 8, 9, 10]
onehotencoder = ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough')
data_transformed = ct.fit_transform(previsores).toarray()

In [80]:
regressor = Sequential()
regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
regressor.add(Dense(units = 158, activation = 'relu'))
regressor.add(Dense(units = 1, activation = 'linear'))
regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mean_about_error'])

c:\Users\samue\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [83]:
regressor.fit(previsores, preco_real, batch_size = 10, epochs = 100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).